In [ ]:
!pip install qiskit==1.4.2
!pip install qiskit_machine_learning==0.8.2
!pip install qiskit_algorithms==0.3.0
!pip install openpyxl
!pip install XlsxWriter

In [ ]:
import warnings, time
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from qiskit.circuit.library import ZFeatureMap, ZZFeatureMap
from qiskit_machine_learning.kernels import FidelityStatevectorKernel

warnings.filterwarnings("ignore")


# ====== CONFIG ======
NUM_REPS      = 30
TEST_SIZE     = 0.3
C_LIST        = [0.1, 1, 10]
SCALES        = np.logspace(-3, 0.3, 8)
CV_CLASSIC_K  = 3
CV_QUANT_K    = 3
FEATURE_DIM   = 4
SEED          = 12345
np.random.seed(SEED)


def load_dataset():
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    mask = (y == 0) | (y == 2)
    Xb = X[mask]
    yb = np.where(y[mask] == 0, -1, 1)

    sss = StratifiedShuffleSplit(
        n_splits=NUM_REPS,
        test_size=TEST_SIZE,
        random_state=SEED
    )
    splits = []
    for tr_idx, ts_idx in sss.split(Xb, yb):
        splits.append((Xb[tr_idx], Xb[ts_idx], yb[tr_idx], yb[ts_idx]))
    return splits


def get_classical_models():
    gammas = np.logspace(-3, 1, 5)
    return {
        "LogReg":   ( LogisticRegression(max_iter=1000), {"C": C_LIST} ),
        "SVM-lin":  ( SVC(kernel="linear"),              {"C": C_LIST} ),
        "SVM-poly": ( SVC(kernel="poly"),   {"degree":[3,4,5],"C":C_LIST,"coef0":[0,1]} ),
        "SVM-rbf":  ( SVC(kernel="rbf"),    {"C":C_LIST,"gamma":gammas} ),
    }


def build_feature_maps():
    maps = []
    # ZFeatureMap reps 1,2
    for r in [1,2]:
        maps.append((f"Z-reps{r}", ZFeatureMap(FEATURE_DIM, reps=r)))
    # ZZFeatureMap reps 1,2 con entanglement linear y full
    for ent in ["linear", "full"]:
        for r in [1,2]:
            maps.append((f"ZZ-{ent}-r{r}", ZZFeatureMap(FEATURE_DIM, reps=r, entanglement=ent)))
    return maps


def main():
    classical_records = []
    quantum_records   = []
    splits = load_dataset()

    for rep, (Xtr, Xts, ytr, yts) in enumerate(splits, 1):
        print(f"--- Repetition {rep}/{NUM_REPS} ---")
        # ----- Modelos clásicos -----
        for name, (clf, grid) in get_classical_models().items():
            t0 = time.time()
            gs = GridSearchCV(clf, grid, cv=CV_CLASSIC_K, scoring="accuracy", n_jobs=-1)
            gs.fit(Xtr, ytr)
            best = gs.best_estimator_
            ypred = best.predict(Xts)
            t_eval = time.time() - t0

            classical_records.append({
                "model":    name,
                "accuracy": accuracy_score(yts, ypred),
                "f1":       f1_score(yts, ypred),
                "auc":      roc_auc_score(yts, ypred),
                "time_s":   round(t_eval,2)
            })

        # ----- Kernels cuánticos -----
        # pre-escalado base [0, π]
        scaler = MinMaxScaler((0, np.pi))
        Xtr0, Xts0 = scaler.fit_transform(Xtr), scaler.transform(Xts)

        for fmap_name, fmap in build_feature_maps():
            for idx, scale in enumerate(SCALES, start=1):
                t0 = time.time()

                # aplicar escala (numérica)
                Xtr_s = Xtr0 * scale
                Xts_s = Xts0 * scale

                # kernel exacto statevector
                qk = FidelityStatevectorKernel(feature_map=fmap)
                qk._validate_input = lambda x, y=None: (x, y)

                Ktr = qk.evaluate(Xtr_s)
                Kts = qk.evaluate(Xts_s, Xtr_s)

                # GridSearchCV para C
                svc = GridSearchCV(
                    SVC(kernel="precomputed"),
                    {"C": C_LIST},
                    cv=CV_QUANT_K,
                    scoring="accuracy"
                )
                svc.fit(Ktr, ytr)
                ypred_q = svc.predict(Kts)
                t_eval = time.time() - t0

                quantum_records.append({
                    "feature_map": f"{fmap_name}-scale{idx}",
                    "scale":       scale,
                    "accuracy":    accuracy_score(yts, ypred_q),
                    "f1":          f1_score(yts, ypred_q),
                    "auc":         roc_auc_score(yts, ypred_q),
                    "time_s":      round(t_eval,2)
                })

    # ----- Agregación -----
    df_cl = pd.DataFrame(classical_records)
    df_q  = pd.DataFrame(quantum_records)

    agg_cl = df_cl.groupby("model").agg(
        acc_mean = ("accuracy", "mean"),
        acc_std  = ("accuracy", "std"),
        f1_mean  = ("f1",       "mean"),
        f1_std   = ("f1",       "std"),
        time_s = ("time_s", "mean")
    ).reset_index().sort_values("acc_mean", ascending=False)

    agg_q = df_q.groupby("feature_map").agg(
        acc_mean = ("accuracy", "mean"),
        acc_std  = ("accuracy", "std"),
        f1_mean  = ("f1",       "mean"),
        f1_std   = ("f1",       "std"),
        scale = ("scale", "mean"),
        time_s = ("time_s", "mean")

    ).reset_index().sort_values("acc_mean", ascending=False)

    print("\n Clásicos (mean ± std):")
    print(agg_cl.to_string(index=False))

    print("\n Cuánticos (mean ± std):")
    print(agg_q.to_string(index=False))

    # Guardar en Excel
    with pd.ExcelWriter("Resultados_IRIS_LINEAR.xlsx") as writer:
        agg_cl.to_excel(writer, sheet_name="Clasicos", index=False)
        agg_q.to_excel(writer, sheet_name="Cuanticos", index=False)

    print("\nResultados_IRIS_LINEAR")

if __name__ == "__main__":
    main()
